In [30]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import re
import io

IndentationError: unexpected indent (<ipython-input-30-d9a7f6f8b1fa>, line 11)

In [18]:

wiki_url = 'https://en.wikipedia.org/wiki/List_of_Moscow_Metro_stations'
wiki_page = requests.get(wiki_url).text
wiki_doc = BeautifulSoup(wiki_page, 'lxml')

stations_table = wiki_doc.find('table', {'class': 'wikitable sortable'})

In [21]:


# Preparation of the list
indices = [0,6]

stations_df = pd.DataFrame(columns=['Line','English name','Coordinates'])

for tr in stations_table.find_all('tr')[1:]:
    line = re.search('#(.+?)\s', tr.find('a')['title']).group(1)
    cells = tr.find_all('td')
    #ignore cells that don't have coordinates
    if cells[6].text.strip() != '':
        stations_df = stations_df.append({
            'Line': line,
            'English name': cells[0].text.strip(),
            'Coordinates': cells[6].find('span', {'class': 'geo'}).text.strip().replace('; ',',')
        }, ignore_index=True)    
        
stations_df.head()


In [23]:
# Line color codes
line_colors = {
    '1':'#EF161E',
    '2':'#2DBE2C',
    '3':'#0078BE',
    '4':'#00BFFF',
    '5':'#8D5B2D',
    '6':'#ED9121',
    '7':'#800080',
    '8':'#FFD702',
    '8A':'#FFD702',
    '9':'#999999',
    '10':'#99CC00',
    '11':'#82C0C0',
    '11A':'#82C0C0',
    '12':'#A1B3D4',
    '13':'#9999FF',
    '14':'#FFFFFF',
    '15':'#DE64A1'
}

#create map of Moscow with all stations
map_moscow_metro = folium.Map(location=[55.755825, 37.617298], zoom_start=10)
#add markers
for line, station, coordinates in zip(stations_df['Line'], stations_df['English name'], stations_df['Coordinates']):
    latlong = [float(x) for x in coordinates.split(',')]
    #Also add a 500-meter circle around the station to visualize our neighborhoods
    #folium.Circle(
    #    latlong,        
    #    radius=1000,
    #    color=line_colors[line]
    #).add_to(map_moscow_metro)
    #Add marker with popup
    folium.Circle(
        latlong,
        popup=station,
        radius=20,
        color=line_colors[line]
    ).add_to(map_moscow_metro)
    if line != prev_line:
        prev_coordinates = None    
    if prev_coordinates != None:
        folium.PolyLine([prev_coordinates, latlong], color=line_colors[line]).add_to(map_moscow_metro)    
    prev_line = line
    prev_coordinates = latlong 
map_moscow_metro

NameError: name 'folium' is not defined

In [31]:
CLIENT_ID = 'KJ5LEKZPM4DFFMCWWZ2OEZIOXWUG1Y2W255PP2RJS4OF3WKF' 
CLIENT_SECRET = 'VTYDQOBB0TNKQ21K4NH5R5PD10JIIU12VNBVQ41JABYCY1WS' 
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, coordinates, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, coordinates):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&{},&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            coordinates, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['coordinates'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Coordinates', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
moscow_venues = getNearbyVenues(names=stations_df['English name'],
                                   coordinates=stations_df['Coordinates']
                                  )